In [1]:
# Cell 1: Import necessary libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from IPython.display import display, Markdown
import warnings
import joblib
import os

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# Cell 2: Load Data (No changes here)
try:
    qualifying_df = pd.read_csv(
        "../data/processed/updated_qualifying.csv", parse_dates=["date"]
    )
    races_df = pd.read_csv(
        "../data/processed/updated_races.csv", parse_dates=["date"]
    )
    print("CSV files loaded successfully.")
except FileNotFoundError as e:
    print(f"Error: File not found. {e}")
    raise

# --- Basic Data Cleaning (No changes here) ---
print("Applying basic data cleaning...")
races_df["points"] = pd.to_numeric(races_df["points"], errors="coerce").fillna(
    0
)
races_df["position"] = pd.to_numeric(races_df["position"], errors="coerce")
races_df["grid"] = pd.to_numeric(races_df["grid"], errors="coerce")


def clean_driver_name(name):
    if isinstance(name, str):
        name = (
            name.replace(" Jr.", "")
            .replace("Hülkenberg", "Hulkenberg")
            .replace("Perez", "Pérez")
            .replace("Raikkonen", "Räikkönen")
        )
        if "Antonelli" in name:
            return "Kimi Antonelli"
    return name


races_df["driverFullName"] = races_df["driverFullName"].apply(clean_driver_name)
qualifying_df["driverFullName"] = qualifying_df["driverFullName"].apply(
    clean_driver_name
)
id_cols = ["driverId", "constructorId"]
for col in id_cols:
    if col in races_df.columns:
        races_df[col] = races_df[col].astype(str)
    if col in qualifying_df.columns:
        qualifying_df[col] = qualifying_df[col].astype(str)
print("Basic data cleaning complete.")

CSV files loaded successfully.
Applying basic data cleaning...
Basic data cleaning complete.


In [3]:
# Cell 3: ADVANCED Feature Engineering Function

def calculate_features_advanced(df_races):
    """
    Calculates advanced features using only historical data.
    Includes:
    - Rolling performance metrics (points, position, grid)
    - Championship standings
    - Driver-Circuit Affinity
    - Teammate Performance Comparison
    - Positions Gained/Lost
    - Standardized Weather Conditions
    """
    print("Calculating ADVANCED features...")
    df = df_races.sort_values(by=["season", "round", "date"]).copy()

    # --- Target Variable & Basic Handling ---
    df["is_winner"] = (df["position"] == 1).astype(int)
    df["grid"] = df["grid"].replace(0, 21).fillna(21)
    df["grid"] = df["grid"].astype(int)

    # --- Base Rolling Performance Metrics (per driver) ---
    df = df.sort_values(by=["driverId", "season", "round", "date"])
    rolling_features_cols = ["points", "position", "grid"]
    for feature_col in rolling_features_cols:
        roll_mean = (
            df.groupby("driverId")[feature_col]
            .rolling(window=5, min_periods=1)
            .mean()
        )
        df[f"avg_{feature_col}_last_5"] = (
            roll_mean.groupby(level=0).shift(1).reset_index(level=0, drop=True)
        )

    # --- Championship Standings (Points Before Race) ---
    df["season_points"] = df.groupby(["season", "driverId"])["points"].cumsum()
    df["points_standings_prev_race"] = (
        df.groupby(["season", "driverId"])["season_points"].shift(1)
    )

    # --- FEATURE 1: Driver-Circuit Affinity ---
    # Calculates a driver's average finish at a specific circuit from past races.
    circuit_finish = (
        df.groupby(["driverId", "circuitId"])["position"]
        .expanding(min_periods=1)
        .mean()
    )
    df["avg_finish_at_circuit"] = (
        circuit_finish.groupby(level=[0, 1])
        .shift(1)
        .reset_index(level=[0, 1], drop=True)
    )

    # --- FEATURE 2: Teammate Comparison ---
    # Compares a driver's recent points haul to their teammate's.
    # First, calculate the team's total average points for each race.
    df["team_total_avg_points"] = df.groupby(
        ["season", "round", "constructorId"]
    )["avg_points_last_5"].transform("sum")
    # Then, find the teammate's score and calculate the difference.
    teammate_avg_points = df["team_total_avg_points"] - df["avg_points_last_5"]
    df["points_diff_vs_teammate_last_5"] = (
        df["avg_points_last_5"] - teammate_avg_points
    )

    # --- FEATURE 3: Qualifying vs. Race Pace (Positions Gained/Lost) ---
    # Calculates a driver's tendency to gain or lose positions in a race.
    df["positions_gained"] = df["grid"] - df["position"]
    roll_gained = (
        df.groupby("driverId")["positions_gained"]
        .rolling(window=5, min_periods=1)
        .mean()
    )
    df["avg_positions_gained_last_5"] = (
        roll_gained.groupby(level=0).shift(1).reset_index(level=0, drop=True)
    )

    # --- FEATURE 4: Standardized Weather Data ---
    if "weather" in df.columns:
        weather_map = {
            "sunny": "dry",
            "clear": "dry",
            "cloudy": "dry",
            "dry": "dry",
            "wet": "wet",
            "rain": "wet",
            "damp": "wet",
            "showers": "wet",
        }
        # Standardize by finding the keyword in the string
        df["weather_condition"] = "unknown"
        for keyword, condition in weather_map.items():
            df.loc[
                df["weather"].str.lower().str.contains(keyword, na=False),
                "weather_condition",
            ] = condition
    else:
        df["weather_condition"] = "unknown"

    # --- Handle NaNs created by shift/rolling for all features ---
    fill_values = {
        "avg_points_last_5": 0,
        "avg_position_last_5": 21,
        "avg_grid_last_5": 21,
        "points_standings_prev_race": 0,
        "avg_finish_at_circuit": 21,  # Default to last place if no history
        "points_diff_vs_teammate_last_5": 0,  # Default to equal if no history
        "avg_positions_gained_last_5": 0,  # Default to neutral if no history
    }
    df.fillna(fill_values, inplace=True)

    # --- Clean up temporary columns ---
    df = df.drop(
        columns=["season_points", "team_total_avg_points", "positions_gained"],
        errors="ignore",
    )
    print("Advanced features calculated successfully.")
    return df.sort_values(by=["season", "round", "date"])


# --- Execute Feature Engineering on Initial Data ---
data_df_featured = calculate_features_advanced(races_df.copy())

# --- Create Name -> ID maps (No changes here) ---
latest_driver_entries = data_df_featured.drop_duplicates(
    subset="driverFullName", keep="last"
)
latest_driver_name_to_id_map = latest_driver_entries.set_index(
    "driverFullName"
)["driverId"].to_dict()
latest_constructor_entries = data_df_featured.drop_duplicates(
    subset="constructorName", keep="last"
)
latest_constructor_name_to_id_map = latest_constructor_entries.set_index(
    "constructorName"
)["constructorId"].to_dict()
print(
    "\nName -> ID maps created from processed data for prediction input handling."
)

Calculating ADVANCED features...
Advanced features calculated successfully.

Name -> ID maps created from processed data for prediction input handling.


In [4]:
# Cell 4: Model Definition and Preprocessing Setup (UPDATED FOR NEW FEATURES)

# Define the full list of features for the model
features = [
    "grid",
    "circuitId",
    "driverId",
    "constructorId",
    "avg_points_last_5",
    "avg_position_last_5",
    "avg_grid_last_5",
    "points_standings_prev_race",
    "avg_finish_at_circuit",  # <-- NEW
    "points_diff_vs_teammate_last_5",  # <-- NEW
    "avg_positions_gained_last_5",  # <-- NEW
    "weather_condition",  # <-- NEW
]
target = "is_winner"

# Define numerical and categorical features FOR THE PREPROCESSOR
numerical_features = [
    "grid",
    "avg_points_last_5",
    "avg_position_last_5",
    "avg_grid_last_5",
    "points_standings_prev_race",
    "avg_finish_at_circuit",  # <-- NEW
    "points_diff_vs_teammate_last_5",  # <-- NEW
    "avg_positions_gained_last_5",  # <-- NEW
]
categorical_features = [
    "circuitId",
    "driverId",
    "constructorId",
    "weather_condition",  # <-- NEW
]

# Create preprocessing pipelines (No changes to the logic)
numerical_transformer = SimpleImputer(strategy="median")
categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        (
            "onehot",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
        ),
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

print("Preprocessor configured with ADVANCED features.")
print("Features for model:", features)

Preprocessor configured with ADVANCED features.
Features for model: ['grid', 'circuitId', 'driverId', 'constructorId', 'avg_points_last_5', 'avg_position_last_5', 'avg_grid_last_5', 'points_standings_prev_race', 'avg_finish_at_circuit', 'points_diff_vs_teammate_last_5', 'avg_positions_gained_last_5', 'weather_condition']


In [5]:
# Cell 5: Hyperparameter Tuning and Model Training (No changes to this logic)
# This cell will now run the tuning process on the new, richer feature set.

X = data_df_featured[features]
y = data_df_featured[target]

gbc_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", GradientBoostingClassifier(random_state=42)),
    ]
)
xgb_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        (
            "classifier",
            XGBClassifier(
                random_state=42, use_label_encoder=False, eval_metric="logloss"
            ),
        ),
    ]
)

param_grid_gbc = {
    "classifier__n_estimators": [100, 200],
    "classifier__learning_rate": [0.05, 0.1],
    "classifier__max_depth": [3, 4],
}
param_grid_xgb = {
    "classifier__n_estimators": [100, 200],
    "classifier__learning_rate": [0.05, 0.1],
    "classifier__max_depth": [3, 4],
    "classifier__subsample": [0.8, 1.0],
    "classifier__colsample_bytree": [0.8, 1.0],
}

print("--- Tuning GradientBoostingClassifier on ADVANCED features ---")
gbc_grid_search = GridSearchCV(
    gbc_pipeline,
    param_grid_gbc,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1,
)
gbc_grid_search.fit(X, y)
print("Best GBC parameters found: ", gbc_grid_search.best_params_)
print("Best GBC ROC AUC score: ", gbc_grid_search.best_score_)

print("\n--- Tuning XGBClassifier on ADVANCED features ---")
xgb_grid_search = GridSearchCV(
    xgb_pipeline,
    param_grid_xgb,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1,
    verbose=1,
)
xgb_grid_search.fit(X, y)
print("Best XGBoost parameters found: ", xgb_grid_search.best_params_)
print("Best XGBoost ROC AUC score: ", xgb_grid_search.best_score_)

if xgb_grid_search.best_score_ > gbc_grid_search.best_score_:
    print("\n--- XGBoost is the winner! Selecting it for predictions. ---")
    best_model = xgb_grid_search.best_estimator_
    model_filename = "joblogs/f1_winner_predictor_model_best_xgb_advanced.joblib"
else:
    print(
        "\n--- GradientBoostingClassifier is the winner! Selecting it for predictions. ---"
    )
    best_model = gbc_grid_search.best_estimator_
    model_filename = "joblogs/f1_winner_predictor_model_best_gbc_advanced.joblib"

try:
    os.makedirs("joblogs", exist_ok=True)
    joblib.dump(best_model, model_filename)
    print(f"Trained champion model saved to {model_filename}")
except Exception as e:
    print(f"Error saving model: {e}")

--- Tuning GradientBoostingClassifier on ADVANCED features ---
Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best GBC parameters found:  {'classifier__learning_rate': 0.05, 'classifier__max_depth': 3, 'classifier__n_estimators': 100}
Best GBC ROC AUC score:  0.9394302693020329

--- Tuning XGBClassifier on ADVANCED features ---
Fitting 5 folds for each of 32 candidates, totalling 160 fits


/Users/daine/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [09:57:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/daine/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [09:57:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/daine/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [09:57:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/daine/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [09:57:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, U

Best XGBoost parameters found:  {'classifier__colsample_bytree': 0.8, 'classifier__learning_rate': 0.05, 'classifier__max_depth': 3, 'classifier__n_estimators': 100, 'classifier__subsample': 0.8}
Best XGBoost ROC AUC score:  0.9446993329431044

--- XGBoost is the winner! Selecting it for predictions. ---
Trained champion model saved to joblogs/f1_winner_predictor_model_best_xgb_advanced.joblib


In [6]:
# %%
# Cell 5.5: Analyze Feature Importances

if "best_model" in locals():
    print("--- Analyzing Feature Importances of the Champion Model ---")

    # The model pipeline has two steps: 'preprocessor' and 'classifier'
    # We need to get the feature names from the 'preprocessor' step
    try:
        # Get the final feature names after one-hot encoding
        feature_names_raw = best_model.named_steps["preprocessor"].get_feature_names_out()

        # Get the importance scores from the 'classifier' step
        importances = best_model.named_steps["classifier"].feature_importances_

        # Create a DataFrame for easy viewing
        importance_df = pd.DataFrame({
            'Feature': feature_names_raw,
            'Importance': importances
        }).sort_values(by='Importance', ascending=False)

        # Clean up the feature names for better readability
        importance_df['Feature'] = importance_df['Feature'].str.replace('num__', '').str.replace('cat__', '')

        print("Top 15 Most Important Features:")
        display(importance_df.head(15))

    except Exception as e:
        print(f"Could not retrieve feature importances. Error: {e}")
        print("This can happen if the model type does not support feature_importances_.")

--- Analyzing Feature Importances of the Champion Model ---
Top 15 Most Important Features:


,Feature,Importance
0,grid,0.187270
1,avg_points_last_5,0.124823
60,driverId_max_verstappen,0.063623
59,driverId_leclerc,0.063030
3,avg_grid_last_5,0.051234
2,avg_position_last_5,0.043599
88,constructorId_red_bull,0.041396
44,driverId_bottas,0.032880
4,points_standings_prev_race,0.030076
66,driverId_piastri,0.027540


In [7]:
# Cell 6: Reusable Grid Preparation Function

TEAM_REBRAND_MAP = {
    "Red Bull Racing Honda RBPT": "red_bull",
    "McLaren Mercedes": "mclaren",
    "McLaren-Mercedes": "mclaren",
    "Ferrari": "ferrari",
    "Mercedes": "mercedes",
    "Racing Bulls Honda RBPT": "rb",
    "Racing Bulls-Honda RBPT": "rb",
    "Williams Mercedes": "williams",
    "Williams-Mercedes": "williams",
    "Haas Ferrari": "haas",
    "Haas-Ferrari": "haas",
    "Alpine Renault": "alpine",
    "Alpine-Renault": "alpine",
    "Aston Martin Aramco Mercedes": "aston_martin",
    "Aston Martin Aramco-Mercedes": "aston_martin",
    "Kick Sauber Ferrari": "sauber",
    "Kick Sauber-Ferrari": "sauber",
    "AlphaTauri": "alphatauri",
    "Racing Point": "racing_point",
    "Alfa Romeo": "alfa",
    "Renault": "renault",
    "RB F1 Team": "rb",
    "Sauber": "sauber",
}


def prepare_grid_for_prediction(
    raw_grid_list,
    driver_name_to_id_map_hist,
    constructor_name_to_id_map_hist,
    team_rebrand_map_current,
):
    """
    Processes a raw grid list, mapping names to IDs and creating placeholders for new entries.
    """
    prepared_grid = []
    print("Preparing grid for prediction...")
    for entry in raw_grid_list:
        driver_name = entry["driver"]
        team_name = entry["team"]
        grid_pos = entry["grid"]

        driver_id = driver_name_to_id_map_hist.get(driver_name)
        if driver_id is None:
            driver_id = f"new_driver_{driver_name.lower().replace(' ', '_')}"
            print(
                f"Note: Using placeholder ID for new/unmapped driver: {driver_name} -> {driver_id}"
            )

        constructor_id_for_model = team_rebrand_map_current.get(team_name)
        if constructor_id_for_model is None:
            constructor_id_for_model = constructor_name_to_id_map_hist.get(
                team_name
            )
        if constructor_id_for_model is None:
            constructor_id_for_model = (
                f"new_team_{team_name.lower().replace(' ', '_')}"
            )
            print(
                f"Note: Using placeholder ID for new/unmapped team: {team_name} -> {constructor_id_for_model}"
            )

        prepared_grid.append(
            {
                "driverId": str(driver_id),
                "constructorId": str(constructor_id_for_model),
                "grid": grid_pos,
                "driverFullName": driver_name,
                "constructorName": team_name,
            }
        )
    print("Grid preparation complete.")
    return prepared_grid

print("Grid preparation function defined.")

Grid preparation function defined.


In [8]:
# Cell 7: Prediction Function Definition
def predict_race_winner_probabilities(
    predict_rows_featured, model, model_features_list, driver_detail_map
):
    """
    Predicts the win probability for each driver in a given grid DataFrame.
    """
    if predict_rows_featured.empty:
        print("Error: predict_rows_featured DataFrame is empty.")
        return {}

    try:
        predict_X = predict_rows_featured[model_features_list]
        probabilities = model.predict_proba(predict_X)
        win_probabilities = probabilities[:, 1]

        total_prob = np.sum(win_probabilities)
        if total_prob > 0:
            normalized_probs = win_probabilities / total_prob
        else:
            print(
                "Warning: Model predicted zero probability for all drivers. Assigning equal probability."
            )
            normalized_probs = np.ones(len(predict_X)) / len(predict_X)

        results = {}
        for i, index in enumerate(predict_rows_featured.index):
            model_driver_id = predict_rows_featured.loc[index, "driverId"]
            details = driver_detail_map.get(model_driver_id)
            if details:
                results[model_driver_id] = {
                    "Probability": normalized_probs[i],
                    "DriverFullName": details["FullName"],
                    "ConstructorName": details["ConstructorName"],
                    "Grid": details["Grid"],
                }
        return dict(
            sorted(
                results.items(),
                key=lambda item: item[1]["Probability"],
                reverse=True,
            )
        )
    except Exception as e:
        print(f"An error occurred during prediction: {e}")
        return {}
    
print("Prediction function updated.")

Prediction function updated.


In [9]:
# %%
# Cell 8: Wrapper Function for Prediction and Display (CORRECTED)

def predict_and_display_results(
    circuit_id,
    future_season,
    future_round,
    raw_grid_list,
    model,
    base_races_df,
    # The base_qualifying_df is no longer needed here as the new function is self-contained
    driver_name_to_id_hist_map,
    constructor_name_to_id_hist_map,
    team_rebrand_map_current,
    model_features_list,
    race_description="Future Race",
    save_path="predictions_BEST_MODEL",
):
    """
    Orchestrates prediction, displays results, and saves them to a CSV.
    Uses the ADVANCED feature calculation function.
    """
    if save_path and not os.path.exists(save_path):
        os.makedirs(save_path)

    print(f"\n--- Predicting for: {race_description} ({circuit_id}) ---")

    prepared_grid_list_with_names = prepare_grid_for_prediction(
        raw_grid_list,
        driver_name_to_id_hist_map,
        constructor_name_to_id_hist_map,
        team_rebrand_map_current,
    )

    driver_detail_map_for_display = {
        item["driverId"]: {
            "FullName": item["driverFullName"],
            "ConstructorName": item["constructorName"],
            "Grid": item["grid"],
        }
        for item in prepared_grid_list_with_names
    }

    future_race_df_for_features = pd.DataFrame(prepared_grid_list_with_names)[
        ["driverId", "constructorId", "grid"]
    ].copy()
    future_race_df_for_features["season"] = future_season
    future_race_df_for_features["round"] = future_round
    future_race_df_for_features["circuitId"] = circuit_id
    future_race_df_for_features["date"] = pd.Timestamp.now()
    future_race_df_for_features["position"] = np.nan
    future_race_df_for_features["points"] = 0.0
    # For future races, we assume dry conditions unless specified otherwise.
    # This ensures the 'weather' column exists for concatenation.
    future_race_df_for_features["weather"] = "dry"

    # Ensure the base DataFrame has the 'weather' column before concatenating
    if "weather" not in base_races_df.columns:
        base_races_df["weather"] = "unknown"

    required_base_cols = [
        "season",
        "round",
        "date",
        "driverId",
        "constructorId",
        "circuitId",
        "grid",
        "position",
        "points",
        "weather",  # <-- Add weather to the list of columns
    ]
    combined_df = pd.concat(
        [base_races_df[required_base_cols], future_race_df_for_features],
        ignore_index=True,
    )

    # *** THIS IS THE FIX: Call the correct function ***
    print("Recalculating features on combined data...")
    combined_featured_df = calculate_features_advanced(combined_df)

    predict_rows_featured = combined_featured_df[
        (combined_featured_df["season"] == future_season)
        & (combined_featured_df["round"] == future_round)
    ].copy()

    if predict_rows_featured.empty:
        print(
            "Error: Could not find rows for the future race after feature calculation."
        )
        return

    winner_probs_dict = predict_race_winner_probabilities(
        predict_rows_featured,
        model,
        model_features_list,
        driver_detail_map_for_display,
    )

    if winner_probs_dict:
        results_df = pd.DataFrame(
            [
                {
                    "Driver": details["DriverFullName"],
                    "Grid": details["Grid"],
                    "Team": details["ConstructorName"],
                    "Probability_Num": details["Probability"],
                }
                for _, details in winner_probs_dict.items()
            ]
        )

        if save_path:
            safe_filename = "".join(
                c if c.isalnum() else "_" for c in race_description
            )
            csv_filename = os.path.join(
                save_path,
                f"{future_season}_R{future_round:02d}_{safe_filename}_{circuit_id}_predictions.csv",
            )
            results_df.to_csv(csv_filename, index=False, float_format="%.6f")
            print(f"Predictions saved to: {csv_filename}")

        display_df = results_df.copy()
        display_df["Probability"] = display_df["Probability_Num"].map(
            "{:.2%}".format
        )
        display_df = display_df.drop(columns=["Probability_Num"])

        markdown_table = "| Driver             | Grid | Team                           | Probability |\n"
        markdown_table += "|--------------------|------|--------------------------------|-------------|\n"
        for _, row in display_df.iterrows():
            markdown_table += (
                f"| {row['Driver']:<18} | {row['Grid']:<4} |"
                f" {row['Team']:<30} | {row['Probability']:>11} |\n"
            )
        display(Markdown(markdown_table))

In [ ]:
# Cell 9: Define all 2025 Race Grids

# The prediction calls will follow in the next cell.

#1
albert_park_2025_raw_grid = [
    # McLaren Mercedes
    {'driver': 'Lando Norris',       'team': 'McLaren Mercedes',              'grid': 1},
    {'driver': 'Oscar Piastri',      'team': 'McLaren Mercedes',              'grid': 2},
    
    # Red Bull Racing Honda RBPT
    {'driver': 'Max Verstappen',     'team': 'Red Bull Racing Honda RBPT',    'grid': 3},
    {'driver': 'Liam Lawson',        'team': 'Red Bull Racing Honda RBPT',    'grid': 18},
    
    # Ferrari
    {'driver': 'Charles Leclerc',    'team': 'Ferrari',                        'grid': 7},
    {'driver': 'Lewis Hamilton',     'team': 'Ferrari',                        'grid': 8},
    
    # Mercedes
    {'driver': 'George Russell',     'team': 'Mercedes',                       'grid': 4},
    {'driver': 'Kimi Antonelli',     'team': 'Mercedes',                       'grid': 16},
    
    # Alpine Renault
    {'driver': 'Pierre Gasly',       'team': 'Alpine Renault',                 'grid': 9},
    {'driver': 'Jack Doohan',        'team': 'Alpine Renault',                 'grid': 14},
    
    # Williams Mercedes
    {'driver': 'Alexander Albon',    'team': 'Williams Mercedes',              'grid': 6},
    {'driver': 'Carlos Sainz',       'team': 'Williams Mercedes',              'grid': 10},
    
    # Racing Bulls Honda RBPT
    {'driver': 'Yuki Tsunoda',       'team': 'Racing Bulls Honda RBPT',        'grid': 5},
    {'driver': 'Isack Hadjar',       'team': 'Racing Bulls Honda RBPT',        'grid': 11},
    
    # Aston Martin Aramco Mercedes
    {'driver': 'Fernando Alonso',    'team': 'Aston Martin Aramco Mercedes',   'grid': 12},
    {'driver': 'Lance Stroll',       'team': 'Aston Martin Aramco Mercedes',   'grid': 13},
    
    # Kick Sauber Ferrari
    {'driver': 'Gabriel Bortoleto',  'team': 'Kick Sauber Ferrari',            'grid': 15},
    {'driver': 'Nico Hulkenberg',    'team': 'Kick Sauber Ferrari',            'grid': 17},
    
    # Haas Ferrari
    {'driver': 'Esteban Ocon',       'team': 'Haas Ferrari',                   'grid': 19},
    {'driver': 'Oliver Bearman',     'team': 'Haas Ferrari',                   'grid': 20},
]

#2
shanghai_2025_raw_grid = [
    # McLaren Mercedes
    {'driver': 'Oscar Piastri',      'team': 'McLaren Mercedes',              'grid': 1},
    {'driver': 'Lando Norris',       'team': 'McLaren Mercedes',              'grid': 3},

    # Mercedes
    {'driver': 'George Russell',     'team': 'Mercedes',                      'grid': 2},
    {'driver': 'Kimi Antonelli',     'team': 'Mercedes',                      'grid': 8},

    # Red Bull Racing Honda RBPT
    {'driver': 'Max Verstappen',     'team': 'Red Bull Racing Honda RBPT',    'grid': 4},
    {'driver': 'Liam Lawson',        'team': 'Red Bull Racing Honda RBPT',    'grid': 20},

    # Ferrari
    {'driver': 'Lewis Hamilton',     'team': 'Ferrari',                       'grid': 5},
    {'driver': 'Charles Leclerc',    'team': 'Ferrari',                       'grid': 6},

    # Racing Bulls Honda RBPT
    {'driver': 'Isack Hadjar',       'team': 'Racing Bulls Honda RBPT',       'grid': 7},
    {'driver': 'Yuki Tsunoda',       'team': 'Racing Bulls Honda RBPT',       'grid': 9},

    # Williams Mercedes
    {'driver': 'Alexander Albon',    'team': 'Williams Mercedes',             'grid': 10},
    {'driver': 'Carlos Sainz',       'team': 'Williams Mercedes',             'grid': 15},

    # Haas Ferrari
    {'driver': 'Esteban Ocon',       'team': 'Haas Ferrari',                  'grid': 11},
    {'driver': 'Oliver Bearman',     'team': 'Haas Ferrari',                  'grid': 17},

    # Kick Sauber Ferrari
    {'driver': 'Nico Hulkenberg',    'team': 'Kick Sauber Ferrari',           'grid': 12},
    {'driver': 'Gabriel Bortoleto',  'team': 'Kick Sauber Ferrari',           'grid': 19},

    # Aston Martin Aramco Mercedes
    {'driver': 'Fernando Alonso',    'team': 'Aston Martin Aramco Mercedes', 'grid': 13},
    {'driver': 'Lance Stroll',       'team': 'Aston Martin Aramco Mercedes', 'grid': 14},

    # Alpine Renault
    {'driver': 'Pierre Gasly',       'team': 'Alpine Renault',                'grid': 16},
    {'driver': 'Jack Doohan',        'team': 'Alpine Renault',                'grid': 18},
]

#3
suzuka_2025_raw_grid = [
    # Red Bull Racing Honda RBPT
    {'driver': 'Max Verstappen',     'team': 'Red Bull Racing Honda RBPT',    'grid': 1},
    {'driver': 'Yuki Tsunoda',       'team': 'Red Bull Racing Honda RBPT',    'grid': 14},

    # McLaren Mercedes
    {'driver': 'Lando Norris',       'team': 'McLaren Mercedes',              'grid': 2},
    {'driver': 'Oscar Piastri',      'team': 'McLaren Mercedes',              'grid': 3},

    # Ferrari
    {'driver': 'Charles Leclerc',    'team': 'Ferrari',                       'grid': 4},
    {'driver': 'Lewis Hamilton',     'team': 'Ferrari',                       'grid': 8},

    # Mercedes
    {'driver': 'George Russell',     'team': 'Mercedes',                      'grid': 5},
    {'driver': 'Kimi Antonelli',     'team': 'Mercedes',                      'grid': 6},

    # Racing Bulls Honda RBPT
    {'driver': 'Isack Hadjar',       'team': 'Racing Bulls Honda RBPT',       'grid': 7},
    {'driver': 'Liam Lawson',        'team': 'Racing Bulls Honda RBPT',       'grid': 13},

    # Williams Mercedes
    {'driver': 'Alexander Albon',    'team': 'Williams Mercedes',             'grid': 9},
    {'driver': 'Carlos Sainz',       'team': 'Williams Mercedes',             'grid': 15},

    # Haas Ferrari
    {'driver': 'Oliver Bearman',     'team': 'Haas Ferrari',                  'grid': 10},
    {'driver': 'Esteban Ocon',       'team': 'Haas Ferrari',                  'grid': 18},

    # Alpine Renault
    {'driver': 'Pierre Gasly',       'team': 'Alpine Renault',                'grid': 11},
    {'driver': 'Jack Doohan',        'team': 'Alpine Renault',                'grid': 19},

    # Aston Martin Aramco Mercedes
    {'driver': 'Fernando Alonso',    'team': 'Aston Martin Aramco Mercedes',  'grid': 12},
    {'driver': 'Lance Stroll',       'team': 'Aston Martin Aramco Mercedes',  'grid': 20},

    # Kick Sauber Ferrari
    {'driver': 'Nico Hulkenberg',    'team': 'Kick Sauber Ferrari',           'grid': 16},
    {'driver': 'Gabriel Bortoleto',  'team': 'Kick Sauber Ferrari',           'grid': 17},
]

#4
bahrain_2025_raw_grid = [
    # McLaren Mercedes
    {'driver': 'Oscar Piastri',      'team': 'McLaren Mercedes',              'grid': 1},
    {'driver': 'Lando Norris',       'team': 'McLaren Mercedes',              'grid': 6},

    # Ferrari
    {'driver': 'Charles Leclerc',    'team': 'Ferrari',                       'grid': 2},
    {'driver': 'Lewis Hamilton',     'team': 'Ferrari',                       'grid': 9},

    # Mercedes
    {'driver': 'George Russell',     'team': 'Mercedes',                      'grid': 3},
    {'driver': 'Kimi Antonelli',     'team': 'Mercedes',                      'grid': 5},

    # Alpine Renault
    {'driver': 'Pierre Gasly',       'team': 'Alpine Renault',                'grid': 4},
    {'driver': 'Jack Doohan',        'team': 'Alpine Renault',                'grid': 11},

    # Red Bull Racing Honda RBPT
    {'driver': 'Max Verstappen',     'team': 'Red Bull Racing Honda RBPT',    'grid': 7},
    {'driver': 'Yuki Tsunoda',       'team': 'Red Bull Racing Honda RBPT',    'grid': 10},

    # Williams Mercedes
    {'driver': 'Carlos Sainz',       'team': 'Williams Mercedes',             'grid': 8},
    {'driver': 'Alexander Albon',    'team': 'Williams Mercedes',             'grid': 15},

    # Racing Bulls Honda RBPT
    {'driver': 'Isack Hadjar',       'team': 'Racing Bulls Honda RBPT',       'grid': 12},
    {'driver': 'Liam Lawson',        'team': 'Racing Bulls Honda RBPT',       'grid': 17},

    # Aston Martin Aramco Mercedes
    {'driver': 'Fernando Alonso',    'team': 'Aston Martin Aramco Mercedes',  'grid': 13},
    {'driver': 'Lance Stroll',       'team': 'Aston Martin Aramco Mercedes',  'grid': 19},

    # Haas Ferrari
    {'driver': 'Esteban Ocon',       'team': 'Haas Ferrari',                  'grid': 14},
    {'driver': 'Oliver Bearman',     'team': 'Haas Ferrari',                  'grid': 20},

    # Kick Sauber Ferrari
    {'driver': 'Nico Hulkenberg',    'team': 'Kick Sauber Ferrari',           'grid': 16},
    {'driver': 'Gabriel Bortoleto',  'team': 'Kick Sauber Ferrari',           'grid': 18},
]

#5
jeddah_2025_raw_grid = [
    # Red Bull Racing Honda RBPT
    {'driver': 'Max Verstappen',     'team': 'Red Bull Racing Honda RBPT',    'grid': 1},
    {'driver': 'Yuki Tsunoda',       'team': 'Red Bull Racing Honda RBPT',    'grid': 8},

    # McLaren Mercedes
    {'driver': 'Oscar Piastri',      'team': 'McLaren Mercedes',              'grid': 2},
    {'driver': 'Lando Norris',       'team': 'McLaren Mercedes',              'grid': 10},

    # Mercedes
    {'driver': 'George Russell',     'team': 'Mercedes',                      'grid': 3},
    {'driver': 'Kimi Antonelli',     'team': 'Mercedes',                      'grid': 5},

    # Ferrari
    {'driver': 'Charles Leclerc',    'team': 'Ferrari',                       'grid': 4},
    {'driver': 'Lewis Hamilton',     'team': 'Ferrari',                       'grid': 7},

    # Williams Mercedes
    {'driver': 'Carlos Sainz',       'team': 'Williams Mercedes',             'grid': 6},
    {'driver': 'Alexander Albon',    'team': 'Williams Mercedes',             'grid': 11},

    # Alpine Renault
    {'driver': 'Pierre Gasly',       'team': 'Alpine Renault',                'grid': 9},
    {'driver': 'Jack Doohan',        'team': 'Alpine Renault',                'grid': 17},

    # Racing Bulls Honda RBPT
    {'driver': 'Liam Lawson',        'team': 'Racing Bulls Honda RBPT',       'grid': 12},
    {'driver': 'Isack Hadjar',       'team': 'Racing Bulls Honda RBPT',       'grid': 14},

    # Aston Martin Aramco Mercedes
    {'driver': 'Fernando Alonso',    'team': 'Aston Martin Aramco Mercedes',  'grid': 13},
    {'driver': 'Lance Stroll',       'team': 'Aston Martin Aramco Mercedes',  'grid': 16},

    # Haas Ferrari
    {'driver': 'Oliver Bearman',     'team': 'Haas Ferrari',                  'grid': 15},
    {'driver': 'Esteban Ocon',       'team': 'Haas Ferrari',                  'grid': 19},

    # Kick Sauber Ferrari
    {'driver': 'Nico Hulkenberg',    'team': 'Kick Sauber Ferrari',           'grid': 18},
    {'driver': 'Gabriel Bortoleto',  'team': 'Kick Sauber Ferrari',           'grid': 20},
]

#6
miami_2025_raw_grid = [
    # Red Bull Racing Honda RBPT
    {'driver': 'Max Verstappen',     'team': 'Red Bull Racing Honda RBPT',    'grid': 1},
    {'driver': 'Yuki Tsunoda',       'team': 'Red Bull Racing Honda RBPT',    'grid': 10},

    # McLaren Mercedes
    {'driver': 'Lando Norris',       'team': 'McLaren Mercedes',              'grid': 2},
    {'driver': 'Oscar Piastri',      'team': 'McLaren Mercedes',              'grid': 4},

    # Mercedes
    {'driver': 'Kimi Antonelli',     'team': 'Mercedes',                      'grid': 3},
    {'driver': 'George Russell',     'team': 'Mercedes',                      'grid': 5},

    # Ferrari
    {'driver': 'Charles Leclerc',    'team': 'Ferrari',                       'grid': 8},
    {'driver': 'Lewis Hamilton',     'team': 'Ferrari',                       'grid': 12},

    # Williams Mercedes
    {'driver': 'Carlos Sainz',       'team': 'Williams Mercedes',             'grid': 6},
    {'driver': 'Alexander Albon',    'team': 'Williams Mercedes',             'grid': 7},

    # Alpine Renault
    {'driver': 'Pierre Gasly',       'team': 'Alpine Renault',                'grid': 20},
    {'driver': 'Jack Doohan',        'team': 'Alpine Renault',                'grid': 14},

    # Racing Bulls Honda RBPT
    {'driver': 'Isack Hadjar',       'team': 'Racing Bulls Honda RBPT',       'grid': 11},
    {'driver': 'Liam Lawson',        'team': 'Racing Bulls Honda RBPT',       'grid': 15},

    # Aston Martin Aramco Mercedes
    {'driver': 'Fernando Alonso',    'team': 'Aston Martin Aramco Mercedes',  'grid': 17},
    {'driver': 'Lance Stroll',       'team': 'Aston Martin Aramco Mercedes',  'grid': 18},

    # Kick Sauber Ferrari
    {'driver': 'Gabriel Bortoleto',  'team': 'Kick Sauber Ferrari',           'grid': 13},
    {'driver': 'Nico Hulkenberg',    'team': 'Kick Sauber Ferrari',           'grid': 16},

    # Haas Ferrari
    {'driver': 'Esteban Ocon',       'team': 'Haas Ferrari',                  'grid': 9},
    {'driver': 'Oliver Bearman',     'team': 'Haas Ferrari',                  'grid': 19},
]

#7
imola_2025_raw_grid = [
    # McLaren Mercedes
    {'driver': 'Oscar Piastri',      'team': 'McLaren Mercedes',              'grid': 1},
    {'driver': 'Lando Norris',       'team': 'McLaren Mercedes',              'grid': 4},

    # Red Bull Racing Honda RBPT
    {'driver': 'Max Verstappen',     'team': 'Red Bull Racing Honda RBPT',    'grid': 2},
    {'driver': 'Yuki Tsunoda',       'team': 'Red Bull Racing Honda RBPT',    'grid': 20},

    # Mercedes
    {'driver': 'George Russell',     'team': 'Mercedes',                      'grid': 3},
    {'driver': 'Kimi Antonelli',     'team': 'Mercedes',                      'grid': 13},

    # Aston Martin Aramco Mercedes
    {'driver': 'Fernando Alonso',    'team': 'Aston Martin Aramco Mercedes',  'grid': 5},
    {'driver': 'Lance Stroll',       'team': 'Aston Martin Aramco Mercedes',  'grid': 8},

    # Williams Mercedes
    {'driver': 'Carlos Sainz',       'team': 'Williams Mercedes',             'grid': 6},
    {'driver': 'Alexander Albon',    'team': 'Williams Mercedes',             'grid': 7},

    # Racing Bulls Honda RBPT
    {'driver': 'Isack Hadjar',       'team': 'Racing Bulls Honda RBPT',       'grid': 9},
    {'driver': 'Liam Lawson',        'team': 'Racing Bulls Honda RBPT',       'grid': 15},

    # Alpine Renault
    {'driver': 'Pierre Gasly',       'team': 'Alpine Renault',                'grid': 10},
    {'driver': 'Franco Colapinto',   'team': 'Alpine Renault',                'grid': 16},  # Time not listed

    # Ferrari
    {'driver': 'Charles Leclerc',    'team': 'Ferrari',                       'grid': 11},
    {'driver': 'Lewis Hamilton',     'team': 'Ferrari',                       'grid': 12},

    # Kick Sauber Ferrari
    {'driver': 'Gabriel Bortoleto',  'team': 'Kick Sauber Ferrari',           'grid': 14},
    {'driver': 'Nico Hulkenberg',    'team': 'Kick Sauber Ferrari',           'grid': 17},

    # Haas Ferrari
    {'driver': 'Esteban Ocon',       'team': 'Haas Ferrari',                  'grid': 18},
    {'driver': 'Oliver Bearman',     'team': 'Haas Ferrari',                  'grid': 19},
]

#8
monaco_2025_raw_grid = [
    # McLaren Mercedes
    {'driver': 'Lando Norris',       'team': 'McLaren Mercedes',              'grid': 1},
    {'driver': 'Oscar Piastri',      'team': 'McLaren Mercedes',              'grid': 3},

    # Ferrari
    {'driver': 'Charles Leclerc',    'team': 'Ferrari',                       'grid': 2},
    {'driver': 'Lewis Hamilton',     'team': 'Ferrari',                       'grid': 7},

    # Red Bull Racing Honda RBPT
    {'driver': 'Max Verstappen',     'team': 'Red Bull Racing Honda RBPT',    'grid': 4},
    {'driver': 'Yuki Tsunoda',       'team': 'Red Bull Racing Honda RBPT',    'grid': 12},
    {'driver': 'Isack Hadjar',       'team': 'Racing Bulls Honda RBPT',       'grid': 5},
    {'driver': 'Liam Lawson',        'team': 'Racing Bulls Honda RBPT',       'grid': 9},

    # Mercedes
    {'driver': 'George Russell',     'team': 'Mercedes',                      'grid': 14},
    {'driver': 'Kimi Antonelli',     'team': 'Mercedes',                      'grid': 15},

    # Aston Martin Aramco Mercedes
    {'driver': 'Fernando Alonso',    'team': 'Aston Martin Aramco Mercedes',  'grid': 6},
    {'driver': 'Lance Stroll',       'team': 'Aston Martin Aramco Mercedes',  'grid': 19},

    # Haas Ferrari
    {'driver': 'Esteban Ocon',       'team': 'Haas Ferrari',                  'grid': 8},
    {'driver': 'Oliver Bearman',     'team': 'Haas Ferrari',                  'grid': 20},

    # Williams Mercedes
    {'driver': 'Alexander Albon',    'team': 'Williams Mercedes',             'grid': 10},
    {'driver': 'Carlos Sainz',       'team': 'Williams Mercedes',             'grid': 11},

    # Kick Sauber Ferrari
    {'driver': 'Nico Hulkenberg',    'team': 'Kick Sauber Ferrari',           'grid': 13},
    {'driver': 'Gabriel Bortoleto',  'team': 'Kick Sauber Ferrari',           'grid': 16},

    # Alpine Renault
    {'driver': 'Pierre Gasly',       'team': 'Alpine Renault',                'grid': 17},
    {'driver': 'Franco Colapinto',   'team': 'Alpine Renault',                'grid': 18},
]

#9
catalunya_2025_raw_grid = [
    # McLaren Mercedes
    {'driver': 'Lando Norris',       'team': 'McLaren Mercedes',              'grid': 2},
    {'driver': 'Oscar Piastri',      'team': 'McLaren Mercedes',              'grid': 1},

    # Ferrari
    {'driver': 'Charles Leclerc',    'team': 'Ferrari',                       'grid': 7},
    {'driver': 'Lewis Hamilton',     'team': 'Ferrari',                       'grid': 5},

    # Red Bull Racing Honda RBPT
    {'driver': 'Max Verstappen',     'team': 'Red Bull Racing Honda RBPT',    'grid': 3},
    {'driver': 'Yuki Tsunoda',       'team': 'Red Bull Racing Honda RBPT',    'grid': 20},
    {'driver': 'Isack Hadjar',       'team': 'Racing Bulls Honda RBPT',       'grid': 9},
    {'driver': 'Liam Lawson',        'team': 'Racing Bulls Honda RBPT',       'grid': 13},

    # Mercedes
    {'driver': 'George Russell',     'team': 'Mercedes',                      'grid': 4},
    {'driver': 'Kimi Antonelli',     'team': 'Mercedes',                      'grid': 6},

    # Aston Martin Aramco Mercedes
    {'driver': 'Fernando Alonso',    'team': 'Aston Martin Aramco Mercedes',  'grid': 10},
    {'driver': 'Lance Stroll',       'team': 'Aston Martin Aramco Mercedes',  'grid': 14},

    # Haas Ferrari
    {'driver': 'Esteban Ocon',       'team': 'Haas Ferrari',                  'grid': 17},
    {'driver': 'Oliver Bearman',     'team': 'Haas Ferrari',                  'grid': 15},

    # Williams Mercedes
    {'driver': 'Alexander Albon',    'team': 'Williams Mercedes',             'grid': 11},
    {'driver': 'Carlos Sainz',       'team': 'Williams Mercedes',             'grid': 18},

    # Kick Sauber Ferrari
    {'driver': 'Nico Hulkenberg',    'team': 'Kick Sauber Ferrari',           'grid': 16},
    {'driver': 'Gabriel Bortoleto',  'team': 'Kick Sauber Ferrari',           'grid': 12},

    # Alpine Renault
    {'driver': 'Pierre Gasly',       'team': 'Alpine Renault',                'grid': 8},
    {'driver': 'Franco Colapinto',   'team': 'Alpine Renault',                'grid': 19},
]

#10
villeneuve_2025_raw_grid = [
    # McLaren Mercedes
    {'driver': 'Lando Norris',       'team': 'McLaren Mercedes',              'grid': 7},
    {'driver': 'Oscar Piastri',      'team': 'McLaren Mercedes',              'grid': 3},

    # Ferrari
    {'driver': 'Charles Leclerc',    'team': 'Ferrari',                       'grid': 8},
    {'driver': 'Lewis Hamilton',     'team': 'Ferrari',                       'grid': 5},

    # Red Bull Racing Honda RBPT
    {'driver': 'Max Verstappen',     'team': 'Red Bull Racing Honda RBPT',    'grid': 2},
    {'driver': 'Yuki Tsunoda',       'team': 'Red Bull Racing Honda RBPT',    'grid': 20},
    {'driver': 'Isack Hadjar',       'team': 'Racing Bulls Honda RBPT',       'grid': 12},
    {'driver': 'Liam Lawson',        'team': 'Racing Bulls Honda RBPT',       'grid': 18},

    # Mercedes
    {'driver': 'George Russell',     'team': 'Mercedes',                      'grid': 1},
    {'driver': 'Kimi Antonelli',     'team': 'Mercedes',                      'grid': 4},

    # Aston Martin Aramco Mercedes
    {'driver': 'Fernando Alonso',    'team': 'Aston Martin Aramco Mercedes',  'grid': 6},
    {'driver': 'Lance Stroll',       'team': 'Aston Martin Aramco Mercedes',  'grid': 17},

    # Haas Ferrari
    {'driver': 'Esteban Ocon',       'team': 'Haas Ferrari',                  'grid': 14},
    {'driver': 'Oliver Bearman',     'team': 'Haas Ferrari',                  'grid': 13},

    # Williams Mercedes
    {'driver': 'Alexander Albon',    'team': 'Williams Mercedes',             'grid': 9},
    {'driver': 'Carlos Sainz',       'team': 'Williams Mercedes',             'grid': 16},

    # Kick Sauber Ferrari
    {'driver': 'Nico Hulkenberg',    'team': 'Kick Sauber Ferrari',           'grid': 11},
    {'driver': 'Gabriel Bortoleto',  'team': 'Kick Sauber Ferrari',           'grid': 15},

    # Alpine Renault
    {'driver': 'Pierre Gasly',       'team': 'Alpine Renault',                'grid': 19},
    {'driver': 'Franco Colapinto',   'team': 'Alpine Renault',                'grid': 10},
]

#11
red_bull_ring_2025_raw_grid = [
    # McLaren Mercedes
    {'driver': 'Lando Norris',       'team': 'McLaren Mercedes',              'grid': 1},
    {'driver': 'Oscar Piastri',      'team': 'McLaren Mercedes',              'grid': 3},

    # Ferrari
    {'driver': 'Charles Leclerc',    'team': 'Ferrari',                       'grid': 2},
    {'driver': 'Lewis Hamilton',     'team': 'Ferrari',                       'grid': 4},

    # Red Bull Racing Honda RBPT
    {'driver': 'Max Verstappen',     'team': 'Red Bull Racing Honda RBPT',    'grid': 7},
    {'driver': 'Yuki Tsunoda',       'team': 'Red Bull Racing Honda RBPT',    'grid': 18},
    {'driver': 'Isack Hadjar',       'team': 'Racing Bulls Honda RBPT',       'grid': 13},
    {'driver': 'Liam Lawson',        'team': 'Racing Bulls Honda RBPT',       'grid': 6},

    # Mercedes
    {'driver': 'George Russell',     'team': 'Mercedes',                      'grid': 5},
    {'driver': 'Kimi Antonelli',     'team': 'Mercedes',                      'grid': 9},

    # Aston Martin Aramco Mercedes
    {'driver': 'Fernando Alonso',    'team': 'Aston Martin Aramco Mercedes',  'grid': 11},
    {'driver': 'Lance Stroll',       'team': 'Aston Martin Aramco Mercedes',  'grid': 16},

    # Haas Ferrari
    {'driver': 'Esteban Ocon',       'team': 'Haas Ferrari',                  'grid': 17},
    {'driver': 'Oliver Bearman',     'team': 'Haas Ferrari',                  'grid': 15},

    # Williams Mercedes
    {'driver': 'Alexander Albon',    'team': 'Williams Mercedes',             'grid': 12},
    {'driver': 'Carlos Sainz',       'team': 'Williams Mercedes',             'grid': 19},

    # Kick Sauber Ferrari
    {'driver': 'Nico Hulkenberg',    'team': 'Kick Sauber Ferrari',           'grid': 20},
    {'driver': 'Gabriel Bortoleto',  'team': 'Kick Sauber Ferrari',           'grid': 8},

    # Alpine Renault
    {'driver': 'Pierre Gasly',       'team': 'Alpine Renault',                'grid': 10},
    {'driver': 'Franco Colapinto',   'team': 'Alpine Renault',                'grid': 14},
]

print("All 2025 race grids defined.")

All 2025 race grids defined.


In [ ]:
# %%
# Cell 10: Execute Predictions for all 2025 Races with the Champion Model (CORRECTED)

# This dictionary holds all the race information.
races_to_predict = {
    "Australian Grand Prix": {
        "circuit_id": "albert_park",
        "season": 2025,
        "round": 1,
        "grid": albert_park_2025_raw_grid,
    },
    "Chinese Grand Prix": {
        "circuit_id": "shanghai",
        "season": 2025,
        "round": 2,
        "grid": shanghai_2025_raw_grid,
    },
    "Japanese Grand Prix": {
        "circuit_id": "suzuka",
        "season": 2025,
        "round": 3,
        "grid": suzuka_2025_raw_grid,
    },
    "Bahrain Grand Prix": {
        "circuit_id": "bahrain",
        "season": 2025,
        "round": 4,
        "grid": bahrain_2025_raw_grid,
    },
    "Saudi Arabian Grand Prix": {
        "circuit_id": "jeddah",
        "season": 2025,
        "round": 5,
        "grid": jeddah_2025_raw_grid,
    },
    "Miami Grand Prix": {
        "circuit_id": "miami",
        "season": 2025,
        "round": 6,
        "grid": miami_2025_raw_grid,
    },
    "Emilia Romagna Grand Prix": {
        "circuit_id": "imola",
        "season": 2025,
        "round": 7,
        "grid": imola_2025_raw_grid,
    },
    "Monaco Grand Prix": {
        "circuit_id": "monaco",
        "season": 2025,
        "round": 8,
        "grid": monaco_2025_raw_grid,
    },
    "Spanish Grand Prix": {
        "circuit_id": "catalunya",
        "season": 2025,
        "round": 9,
        "grid": catalunya_2025_raw_grid,
    },
    "Canadian Grand Prix": {
        "circuit_id": "villeneuve",
        "season": 2025,
        "round": 10,
        "grid": villeneuve_2025_raw_grid,
    },
    "Austrian Grand Prix": {
        "circuit_id": "red_bull_ring",
        "season": 2025,
        "round": 11,
        "grid": red_bull_ring_2025_raw_grid,
    },
}

# Check if the best_model was trained successfully before proceeding
if "best_model" in locals():
    for race_name, race_info in races_to_predict.items():
        predict_and_display_results(
            circuit_id=race_info["circuit_id"],
            future_season=race_info["season"],
            future_round=race_info["round"],
            raw_grid_list=race_info["grid"],
            model=best_model,
            base_races_df=races_df,
            # base_qualifying_df=qualifying_df,
            driver_name_to_id_hist_map=latest_driver_name_to_id_map,
            constructor_name_to_id_hist_map=latest_constructor_name_to_id_map,
            team_rebrand_map_current=TEAM_REBRAND_MAP,
            model_features_list=features,
            race_description=f"2025 {race_name}",
        )
else:
    print(
        "Prediction skipped because the model tuning process did not complete successfully."
    )


--- Predicting for: 2025 Australian Grand Prix (albert_park) ---
Preparing grid for prediction...
Grid preparation complete.
Recalculating features on combined data...
Calculating ADVANCED features...
Advanced features calculated successfully.
Predictions saved to: predictions_BEST_MODEL/2025_R01_2025_Australian_Grand_Prix_albert_park_predictions.csv


| Driver             | Grid | Team                           | Probability |
|--------------------|------|--------------------------------|-------------|
| Lando Norris       | 1    | McLaren Mercedes               |      24.77% |
| Oscar Piastri      | 2    | McLaren Mercedes               |       7.80% |
| Max Verstappen     | 3    | Red Bull Racing Honda RBPT     |       7.31% |
| George Russell     | 4    | Mercedes                       |       5.74% |
| Charles Leclerc    | 7    | Ferrari                        |       0.66% |
| Kimi Antonelli     | 16   | Mercedes                       |       0.42% |
| Lewis Hamilton     | 8    | Ferrari                        |       0.37% |
| Carlos Sainz       | 10   | Williams Mercedes              |       0.22% |
| Alexander Albon    | 6    | Williams Mercedes              |       0.20% |
| Pierre Gasly       | 9    | Alpine Renault                 |       0.20% |
| Yuki Tsunoda       | 5    | Racing Bulls Honda RBPT        |       0.19% |
| Jack Doohan        | 14   | Alpine Renault                 |       0.14% |
| Isack Hadjar       | 11   | Racing Bulls Honda RBPT        |       0.13% |
| Gabriel Bortoleto  | 15   | Kick Sauber Ferrari            |       0.12% |
| Liam Lawson        | 18   | Red Bull Racing Honda RBPT     |       0.12% |
| Esteban Ocon       | 19   | Haas Ferrari                   |       0.11% |
| Fernando Alonso    | 12   | Aston Martin Aramco Mercedes   |       0.11% |
| Oliver Bearman     | 20   | Haas Ferrari                   |       0.10% |
| Lance Stroll       | 13   | Aston Martin Aramco Mercedes   |       0.09% |
| Nico Hulkenberg    | 17   | Kick Sauber Ferrari            |       0.09% |



--- Predicting for: 2025 Chinese Grand Prix (shanghai) ---
Preparing grid for prediction...
Grid preparation complete.
Recalculating features on combined data...
Calculating ADVANCED features...
Advanced features calculated successfully.
Predictions saved to: predictions_BEST_MODEL/2025_R02_2025_Chinese_Grand_Prix_shanghai_predictions.csv


| Driver             | Grid | Team                           | Probability |
|--------------------|------|--------------------------------|-------------|
| Oscar Piastri      | 1    | McLaren Mercedes               |      22.42% |
| George Russell     | 2    | Mercedes                       |      10.54% |
| Max Verstappen     | 4    | Red Bull Racing Honda RBPT     |       9.56% |
| Lando Norris       | 3    | McLaren Mercedes               |       6.93% |
| Charles Leclerc    | 6    | Ferrari                        |       0.57% |
| Kimi Antonelli     | 8    | Mercedes                       |       0.46% |
| Carlos Sainz       | 15   | Williams Mercedes              |       0.36% |
| Lewis Hamilton     | 5    | Ferrari                        |       0.22% |
| Isack Hadjar       | 7    | Racing Bulls Honda RBPT        |       0.18% |
| Yuki Tsunoda       | 9    | Racing Bulls Honda RBPT        |       0.16% |
| Alexander Albon    | 10   | Williams Mercedes              |       0.16% |
| Jack Doohan        | 18   | Alpine Renault                 |       0.12% |
| Pierre Gasly       | 16   | Alpine Renault                 |       0.10% |
| Liam Lawson        | 20   | Red Bull Racing Honda RBPT     |       0.10% |
| Esteban Ocon       | 11   | Haas Ferrari                   |       0.09% |
| Gabriel Bortoleto  | 19   | Kick Sauber Ferrari            |       0.09% |
| Oliver Bearman     | 17   | Haas Ferrari                   |       0.09% |
| Lance Stroll       | 14   | Aston Martin Aramco Mercedes   |       0.08% |
| Fernando Alonso    | 13   | Aston Martin Aramco Mercedes   |       0.08% |
| Nico Hulkenberg    | 12   | Kick Sauber Ferrari            |       0.08% |



--- Predicting for: 2025 Japanese Grand Prix (suzuka) ---
Preparing grid for prediction...
Grid preparation complete.
Recalculating features on combined data...
Calculating ADVANCED features...
Advanced features calculated successfully.
Predictions saved to: predictions_BEST_MODEL/2025_R03_2025_Japanese_Grand_Prix_suzuka_predictions.csv


...

| Driver             | Grid | Team                           | Probability |
|--------------------|------|--------------------------------|-------------|
| Lando Norris       | 1    | McLaren Mercedes               |      21.04% |
| Oscar Piastri      | 3    | McLaren Mercedes               |      10.22% |
| Max Verstappen     | 4    | Red Bull Racing Honda RBPT     |      10.15% |
| Charles Leclerc    | 2    | Ferrari                        |       6.34% |
| Lewis Hamilton     | 7    | Ferrari                        |       0.61% |
| George Russell     | 14   | Mercedes                       |       0.39% |
| Alexander Albon    | 10   | Williams Mercedes              |       0.27% |
| Fernando Alonso    | 6    | Aston Martin Aramco Mercedes   |       0.17% |
| Liam Lawson        | 9    | Racing Bulls Honda RBPT        |       0.15% |
| Isack Hadjar       | 5    | Racing Bulls Honda RBPT        |       0.14% |
| Esteban Ocon       | 8    | Haas Ferrari                   |       0.14% |
| Carlos Sainz       | 11   | Williams Mercedes              |       0.13% |
| Franco Colapinto   | 18   | Alpine Renault                 |       0.12% |
| Kimi Antonelli     | 15   | Mercedes                       |       0.12% |
| Gabriel Bortoleto  | 16   | Kick Sauber Ferrari            |       0.12% |
| Pierre Gasly       | 17   | Alpine Renault                 |       0.10% |
| Nico Hulkenberg    | 13   | Kick Sauber Ferrari            |       0.10% |
| Lance Stroll       | 19   | Aston Martin Aramco Mercedes   |       0.10% |
| Oliver Bearman     | 20   | Haas Ferrari                   |       0.09% |
| Yuki Tsunoda       | 12   | Red Bull Racing Honda RBPT     |       0.09% |



--- Predicting for: 2025 Spanish Grand Prix (catalunya) ---
Preparing grid for prediction...
Grid preparation complete.
Recalculating features on combined data...
Calculating ADVANCED features...
Advanced features calculated successfully.
Predictions saved to: predictions_BEST_MODEL/2025_R09_2025_Spanish_Grand_Prix_catalunya_predictions.csv


| Driver             | Grid | Team                           | Probability |
|--------------------|------|--------------------------------|-------------|
| Oscar Piastri      | 1    | McLaren Mercedes               |      27.27% |
| Lando Norris       | 2    | McLaren Mercedes               |       9.09% |
| Max Verstappen     | 3    | Red Bull Racing Honda RBPT     |       7.61% |
| George Russell     | 4    | Mercedes                       |       2.04% |
| Charles Leclerc    | 7    | Ferrari                        |       0.54% |
| Lewis Hamilton     | 5    | Ferrari                        |       0.49% |
| Kimi Antonelli     | 6    | Mercedes                       |       0.25% |
| Pierre Gasly       | 8    | Alpine Renault                 |       0.16% |
| Isack Hadjar       | 9    | Racing Bulls Honda RBPT        |       0.15% |
| Fernando Alonso    | 10   | Aston Martin Aramco Mercedes   |       0.14% |
| Gabriel Bortoleto  | 12   | Kick Sauber Ferrari            |       0.11% |
| Franco Colapinto   | 19   | Alpine Renault                 |       0.11% |
| Lance Stroll       | 14   | Aston Martin Aramco Mercedes   |       0.10% |
| Oliver Bearman     | 15   | Haas Ferrari                   |       0.10% |
| Yuki Tsunoda       | 20   | Red Bull Racing Honda RBPT     |       0.09% |
| Esteban Ocon       | 17   | Haas Ferrari                   |       0.09% |
| Carlos Sainz       | 18   | Williams Mercedes              |       0.09% |
| Liam Lawson        | 13   | Racing Bulls Honda RBPT        |       0.08% |
| Alexander Albon    | 11   | Williams Mercedes              |       0.08% |
| Nico Hulkenberg    | 16   | Kick Sauber Ferrari            |       0.07% |



--- Predicting for: 2025 Canadian Grand Prix (villeneuve) ---
Preparing grid for prediction...
Grid preparation complete.
Recalculating features on combined data...
Calculating ADVANCED features...
Advanced features calculated successfully.
Predictions saved to: predictions_BEST_MODEL/2025_R10_2025_Canadian_Grand_Prix_villeneuve_predictions.csv


| Driver             | Grid | Team                           | Probability |
|--------------------|------|--------------------------------|-------------|
| George Russell     | 1    | Mercedes                       |      22.27% |
| Max Verstappen     | 2    | Red Bull Racing Honda RBPT     |      11.83% |
| Oscar Piastri      | 3    | McLaren Mercedes               |       9.13% |
| Lando Norris       | 7    | McLaren Mercedes               |       2.56% |
| Lewis Hamilton     | 5    | Ferrari                        |       0.98% |
| Charles Leclerc    | 8    | Ferrari                        |       0.69% |
| Kimi Antonelli     | 4    | Mercedes                       |       0.37% |
| Alexander Albon    | 9    | Williams Mercedes              |       0.20% |
| Franco Colapinto   | 10   | Alpine Renault                 |       0.20% |
| Fernando Alonso    | 6    | Aston Martin Aramco Mercedes   |       0.20% |
| Isack Hadjar       | 12   | Racing Bulls Honda RBPT        |       0.14% |
| Lance Stroll       | 17   | Aston Martin Aramco Mercedes   |       0.13% |
| Gabriel Bortoleto  | 15   | Kick Sauber Ferrari            |       0.12% |
| Yuki Tsunoda       | 20   | Red Bull Racing Honda RBPT     |       0.11% |
| Carlos Sainz       | 16   | Williams Mercedes              |       0.11% |
| Oliver Bearman     | 13   | Haas Ferrari                   |       0.11% |
| Pierre Gasly       | 19   | Alpine Renault                 |       0.10% |
| Esteban Ocon       | 14   | Haas Ferrari                   |       0.10% |
| Liam Lawson        | 18   | Racing Bulls Honda RBPT        |       0.09% |
| Nico Hulkenberg    | 11   | Kick Sauber Ferrari            |       0.09% |
